In [4]:
import json
import xmltodict
import pandas as pd
pd.options.display.max_columns=30000

file_name = '../source/people/source.xml'
save_file = '../source/people/source.json'
languages = [{'0': 'en'}, {'1': 'nl'}, {'2': 'fr'}, {'3': 'de'}, {'4': 'ar'}, {'5': 'it'}, 
             {'6': 'gr'}, {'7': '-'}, {'8': '-'}, {'9': 'sv'}, {'10': 'heb'}, {'11': 'no'}]


with open(file_name, encoding='utf-8') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())
    
    records = data_dict['pockets']['pocket']
#with open(save_file, 'w') as outfile:
#    json.dump(data_dict, outfile, indent=2, ensure_ascii=False)

print('Done')

Done


In [6]:
data = data_dict['pockets']['pocket']

matched_dict=list(filter(lambda data:data['@id']=='211',data))
print(matched_dict[0]['record']['name.type']['value'][0]['#text'])

MAKER


In [9]:

concept_dict = {}
concept_list = []
term_type_list = []

def select_dict(d, lang1, lang2):
    fallback = None
    for item in d:
        if item['@lang'] == lang1:
            return item
        elif item['@lang'] == lang2:
            fallback = item
    return fallback

for pocket in records:
    record = pocket['record']
            
    if 'name' in record.keys():
        concept_dict['selected'] = record['@selected']
        concept_dict['modified'] = record['@modification']
        concept_dict['created'] = record['@created']
        concept_dict['id'] = record['priref']
    

        ## name, check for many
        if 'name' in record.keys():
            if isinstance(record['name']['value'], dict):            
                concept_dict['name'] = record['name']['value']['#text']
                if len(record['name']['value']['@lang']) == 0:
                    concept_dict['name_lang'] = 'nl'
                else:               
                    concept_dict['name_lang'] = record['name']['value']['@lang'][:2]
            elif isinstance(record['name']['value'], list):                
                _term_dict = select_dict(record['name']['value'], 'nl-NL', 'en-US')
                concept_dict['name'] = _term_dict['#text']
                concept_dict['name_lang'] = _term_dict['@lang'][:2]
            concept_dict['name_dict'] = record['name']
            
        ## Term type
        if 'name.type' in record.keys():
            if isinstance(record['name.type'], dict):
                try:
                    _term_type_dict = select_dict(record['name.type']['value'], 'neutral', '0')
                    concept_dict['name_type'] = _term_type_dict['#text']
                except:
                    # 113374 {'@lang': 'neutral', '#text': 'objectnaam'}
                    concept_dict['name_type'] = record['name.type']['value']['#text']
            elif isinstance(record['name.type'], list):
                for record_type in record['name.type']:
                    _term_type_dict = select_dict(record_type['value'], 'neutral', '0')
                    term_type_list.append(_term_type_dict['#text'])
                    _term_type_dict = []
                concept_dict['name_type'] = term_type_list
                term_type_list = []    
            concept_dict['name_type_dict'] = record['name.type']
            
            
        ## Source
        if 'source' in record.keys():
            if isinstance(record['source'], dict):            
                concept_dict['source_name'] = record['source']['source']
                if 'source.number' in record['source'].keys():
                    concept_dict['source_id'] = record['source']['source.number']
                if 'source.url' in record['source']:
                    concept_dict['source_url'] = record['source']['source.url']
            if isinstance(record['source'], list):            
                concept_dict['source_name'] = record['source'][0]['source']
                if 'source.number' in record['source'][0].keys():
                    concept_dict['source_id'] = record['source'][0]['source.number']
                if 'source.url' in record['source'][0]:
                    concept_dict['source_url'] = record['source'][0]['source.url']
            concept_dict['source_dict'] = record['source']

        
        def get_term_type(record_use_lref):
            record_use_list = []
            term_type_list = []
            
            if isinstance(record_use_lref, str):
                record_use_list.append(record_use_lref)
            elif isinstance(record_use_lref, list):
                record_use_list = record_use_lref                        
            for match_record_id in record_use_list:                                                
                matched_record = list(filter(lambda data:data['@id']== match_record_id, data))
                if 'term.type' in matched_record[0]['record'].keys():
                    matched_type = matched_record[0]['record']['term.type']
                    if isinstance(matched_type, dict):
                        term_type_dict = select_dict(matched_type['value'], 'neutral', '0')                                                                         
                        return term_type_dict['#text']                   
                    if isinstance(matched_type, list):                       
                        for term_type in matched_type:                            
                            term_type_dict = select_dict(term_type['value'], 'neutral', '0')                        
                            term_type_list.append(term_type_dict['#text'])
                            term_type_dict = []                            
                        return term_type_list
                        return str(term_type_list)
        
        #if 'use.lref' in record.keys():
        #    term_type = get_term_type(record['use.lref'])
        #    #print(record['priref'], record['use.lref'], term_type)
        #    concept_dict['term_type'] = term_type
        #    concept_dict['use_id'] = record['use.lref']    
        
        
        if 'use' in record.keys():
            concept_dict['use_dict'] = record['use']                
        if 'used_for.lref' in record.keys():
            concept_dict['used_for_id'] = record['used_for.lref']
                
        concept_list.append(concept_dict)
        concept_dict = {}
        #print(json.dumps(concept_list, indent=2))
        #print('_____________________')

concept_df = pd.DataFrame(concept_list)
concept_df.to_csv('out/concepts.csv', index=False)
concept_df

,selected,modified,created,id,name,name_lang,name_dict,name_type,name_type_dict,used_for_id,use_dict
0,False,2020-04-23T13:29:46,2013-11-06T12:43:58,1,onbekend,nl,"{'value': {'@invariant': 'false', '@lang': 'nl...","[INST, PUBL, MAKER, ACQUISITIONSOURCE, VALUER]","[{'value': [{'@lang': 'neutral', '#text': 'INS...",NaN,NaN
1,False,2018-10-10T17:12:47,2008-01-31T09:00:51,2,"Bergh, Henriette Mayer van den",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",MAKER,"{'value': [{'@lang': 'neutral', '#text': 'MAKE...",NaN,NaN
2,False,2022-08-09T14:06:00,2008-01-31T09:00:52,3,"Mayer van den Bergh, Fritz",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...","[PERSON, MAKER]","[{'value': [{'@lang': 'neutral', '#text': 'PER...",NaN,NaN
3,False,2022-06-09T12:38:29,2008-01-31T09:00:53,4,"Touzain, Gilbert",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...","[ACQUISITIONSOURCE, PERSON]","[{'value': [{'@lang': 'neutral', '#text': 'ACQ...",NaN,NaN
4,False,2022-03-04T14:48:33,2008-01-31T08:00:58,5,Museum Mayer van den Bergh (Antwerpen),nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",INST,"{'value': [{'@lang': 'neutral', '#text': 'INST...","[1085, 653]",NaN
...,...,...,...,...,...,...,...,...,...,...,...
32161,False,2022-09-19T15:11:12,2022-09-19T15:10:06,36201,"Caspari, Gertrud",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",MAKER,"{'value': [{'@lang': 'neutral', '#text': 'MAKE...",NaN,NaN
32162,False,2022-09-19T15:47:04,2022-09-19T15:45:38,36202,"Rackham, Arthur",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",MAKER,"{'value': [{'@lang': 'neutral', '#text': 'MAKE...",NaN,NaN
32163,False,2022-09-26T09:44:24,2022-09-26T09:43:04,36203,"Hieronymus, Sophronius Eusebius",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",AUTHOR,"{'value': [{'@lang': 'neutral', '#text': 'AUTH...",NaN,NaN
32164,False,2022-09-26T15:26:56,2022-09-26T15:26:12,36204,"Puget de La Serre, Jean",nl,"{'value': {'@invariant': 'false', '@lang': 'nl...",AUTHOR,"{'value': [{'@lang': 'neutral', '#text': 'AUTH...",NaN,NaN


In [22]:
##[{'@lang': 'neutral', '#text': 'OBJECT'}, {'@lang': '0', '#text': 'object name'}, {'@lang': '1', '#text': 'objectnaam'}, {'@lang': '2', '#text': "nom d'objet"}, {'@lang': '3', '#text': 'Objektbezeichnung'}, {'@lang': '4', '#text': 'إسم_قطعة.فنية'}, {'@lang': '5', '#text': "nome dell'oggetto"}, {'@lang': '6', '#text': 'ονομασία αντικειμ.'}, {'@lang': '9', '#text': 'föremål namn'}, {'@lang': '10', '#text': 'שם האובייקט'}, {'@lang': '11', '#text': 'genstand navn'}]
## [{'@lang': 'neutral', '#text': 'MATER'}, {'@lang': '0', '#text': 'material'}, {'@lang': '1', '#text': 'materiaal'}, {'@lang': '2', '#text': 'matériel'}, {'@lang': '3', '#text': 'Material'}, {'@lang': '4', '#text': 'المواد'}, {'@lang': '5', '#text': 'materiale'}, {'@lang': '6', '#text': 'υλικό'}, {'@lang': '9', '#text': 'material'}, {'@lang': '10', '#text': 'חומר'}, {'@lang': '11', '#text': 'materiale'}]

In [ ]:
key_list = []
for item in data_dict['pockets']['pocket']:    
    item_keys = list(item['record'].keys())    
    key_list.append(item_keys)
keys_df = pd.DataFrame(key_list)
keys_df.drop_duplicates().reset_index().to_csv('out/keys.csv', index=False)